In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import tifffile
from matplotlib.patches import Polygon
from skimage.exposure import adjust_gamma
from mplex import Grid
from mplex.annotate import add_scale_bars
from mplex.text import set_text_outline
from mplex.cm import lerp_colors
from bino_utils import load_config
from bino_utils.plotting import add_anat_axes, add_ocular_condition_symbol

In [2]:
def mask2points(mask, s=50, n_points=50):
    import cv2
    from scipy.interpolate import splprep, splev

    mask = mask.astype(np.uint8)
    contours = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    tck, _ = splprep(contours[0][0][:, 0].T, s=s, per=True)
    x, y = splev(np.linspace(0, 1, n_points, endpoint=False), tck)
    return np.column_stack((x, y))

In [3]:
data_dir = Path("../../data/eye_block")
out_dir = Path("outputs/")
out_dir.mkdir(exist_ok=True)
fmt = ".pdf"

config = load_config()
cond_colors = config["palettes"]["ocular_conditions"]
figh = 100
px_per_um = 0.994368911043582
sb_len = 50 / px_per_um
sb_lw = 0.75

In [4]:
im = tifffile.imread(data_dir / "20220825-F1_session3_plane5_mean_proj.tif")
im = adjust_gamma(im, 0.25)
masks = tifffile.imread(data_dir / "20220825-F1_session3_plane5_af7_masks.tif") != 0
contours = np.array([mask2points(mask) for mask in masks])

In [ ]:
cmap = lerp_colors("k", (0, 1, 0))
vmin = np.quantile(im, 0.1)
vmax = np.quantile(im, 1 - 1e-3)
title = "atoh7:Gal4 UAS:GCaMP6s"
xlim = (39.5, 481.5)
ylim = (441.5, -0.5)
y = contours[..., 1].min()

g = Grid(figh)
ax = g.item()
ax.axis("off")
ax.imshow(im, cmap=cmap, vmin=vmin, vmax=vmax)
ax.set_title(title, style="italic")
ax.set_xlim(xlim)
ax.set_ylim(ylim)

for text, contour in zip(("Left", "Right"), contours):
    ax.add_patch(Polygon(contour, fc="none", ec="w", ls="--"))
    x = contour[contour[:, 1].argmin(), 0]
    s = f"{text} AF7"
    kw = dict(lw=1, c="k")
    ax.add_text(x, y, s, c="w", ha="c", outline_kwargs=kw, pad=(0, 2))

ret = add_anat_axes(sb_len)
x = sum(ax.get_xlim()) - ret["line"].get_xdata().min()
y = ret["line"].get_ydata().max()
ax.plot([x, x - sb_len], [y, y], c="w", lw=sb_lw, solid_capstyle="butt")

if fmt:
    g.savefig((out_dir / "fluorescence_image").with_suffix(fmt), dpi=300)

In [6]:
ims = tifffile.imread(data_dir / "activity_images.tif")
r = np.abs(contours - contours.mean(axis=1, keepdims=True)).max() + 8
extents = np.array([[117, 181, 140, 76], [328, 392, 139, 75]])
qmax = 1 - 1e-3
vmax = 2.5
vmin = 0
cmap = "magma"
space = 1
dff = "norm.\nΔF/F"

In [ ]:
g = Grid((figh - space * 3) / 4, (4, 2), space=space, sharex=False, sharey=False)
g.axis("off")
g[0, 0].set_title("Left")
g[0, 1].set_title("Right")

for i, cond in enumerate(cond_colors):
    add_ocular_condition_symbol(
        cond, 0, 0.5, xpad=-1, ha="r", va="c", scale=0.75, ax=g[i, 0]
    )
    for ax, im, extent, cnt in zip(g[i], ims[i], extents, contours):
        ax.imshow(im, vmin=vmin, vmax=vmax, cmap=cmap, extent=extent)
        ax.add_patch(Polygon(cnt, fc="none", ec="w", ls="--", lw=0.4, alpha=0.8))
        ax.set_xlim(cnt[:, 0].mean() + np.array((-r, r)))
        ax.set_ylim(cnt[:, 1].mean() + np.array((r, -r)))

ax = g[-1, -1]
cb_len = (135 - space * 3) / 8
cb = ax.add_colorbar(
    vmin, vmax, cmap, length=cb_len, pad=2, thick=5, loc0="rb", loc1="lb"
)
cb.set_ticks([0, vmax], labels=[0, vmax])
cb.ax.set_title("N" + dff[1:], size=6, loc="left", pad=4)
cb.ax.tick_params("both", labelsize=6, length=1, pad=1)

x, y = ax.transData.inverted().transform(
    ax.transAxes.transform((1 - 1.5 / g.gridw[-2], 1.5 / g.gridh[-2]))
)
ax.plot([x, x - 50 / px_per_um], [y, y], c="w", lw=sb_lw)

if fmt:
    g.savefig((out_dir / "activity_images").with_suffix(fmt))

In [8]:
df_traces = pd.read_hdf(data_dir / "traces.h5")
dt = 0.3326993332720455
t0 = 9 * dt
t1 = t0 + 2

In [ ]:
g = Grid((figh - space * 3) / 4, (4, 2), space=(2, space))
g.axis("off")
g.set_xmargin(0)
g.set_ymargin(0)
g[0, 0].set_title("Left")
g[0, 1].set_title("Right")

for i, (cond, c) in enumerate(cond_colors.items()):
    for side, ax in zip("lr", g[i]):
        df = df_traces[df_traces["condition"].eq(cond) & df_traces["side"].eq(side)]
        ax.plot(df["t"], df["mean"], lw=sb_lw, color=c)
        ax.fill_between(df["t"], y1=df["low"], y2=df["high"], alpha=0.3, lw=0, color=c)

text_kw = (dict(rotation=0, size=6), dict(rotation=0, ha="right", va="center", size=6))
ret = add_scale_bars(
    t0, 2, 2, 1, "s", dff, (2, 2), sb_lw, text_kw=text_kw, ax=g[-1, -1]
)
set_text_outline(ret["text_x"], lw=2, c="w")

for j in range(2):
    ax = g[:, j].make_ax(sharex=True, behind=True)
    ax.vlines((t0, t1), 0, 1, lw=0.5, ls="--", alpha=0.3, color="k")
    ax.set_ylim(0, 1)

if fmt:
    g.savefig((out_dir / "activities").with_suffix(fmt))

In [10]:
df_tuning = pd.read_hdf(data_dir / "tuning.h5")
df_tuning.iloc[:, -3:] -= (
    df_tuning.loc[df_tuning["condition"].eq("0"), "low"].mean().item()
)

In [ ]:
g = Grid(
    (80, (figh - 30) / 2),
    (2, 1),
    space=(0, 15),
    spines="lb",
    ticks_sides="lb",
    ticklabels_sides="lb",
)
g.set_xmargin(0)
g.set_ymargin(0)

g[0, 0].set_title("Left AF7", size=7)
g[1, 0].set_title("Right AF7", size=7)

for i, (side, ax) in enumerate(zip("lr", g[:, 0])):
    for k, (cond, c) in enumerate(cond_colors.items()):
        df = df_tuning[df_tuning["side"].eq(side) & df_tuning["condition"].eq(cond)]
        ax.plot(df["azimuth_int"], df["mean"], lw=sb_lw, solid_capstyle="butt", color=c)
        ax.fill_between(
            df["azimuth_int"], df["low"], df["high"], alpha=0.2, lw=0, color=c
        )

ax = g[-1, -1]
ax.set_xticks([-40, -20, 0, 20, 40])
ax.set_yticks([0, 2, 4])
ax.set_ylim(None, 4)
ax.set_xlabel("Stimulus azimuth (°)")
text_kw = dict(rotation=0, size=6, va="center", ha="right")
ax = g[:].make_ax()
ax.axis("on")
ax.set_visible_sides(False)
ax.set_ylabel("Max. norm. ΔF/F", labelpad=8)

ax = g[0, 0]
for i, c in enumerate("lrb0"):
    add_ocular_condition_symbol(
        c, 0, 1, ha="l", va="t", scale=0.6, ax=ax, xpad=2, ypad=-i * 6.5
    )

if fmt:
    g.savefig((out_dir / "tuning_curves").with_suffix(fmt))